<table>
    <tr><td align="right" style="background-color:#ffffff;">
        <img src="../images/logo.jpg" width="20%" align="right">
    </td></tr>
    <tr><td align="right" style="color:#777777;background-color:#ffffff;font-size:12px;">
        Prepared by Maksim Dimitrijev and Özlem Salehi <br>
        Özlem Salehi | December 5, 2019 (updated) 
    </td></tr>
    <tr><td align="right" style="color:#bbbbbb;background-color:#ffffff;font-size:11px;font-style:italic;">
        This cell contains some macros. If there is a problem with displaying mathematical formulas, please run this cell to load these macros.
    </td></tr>
</table>
$ \newcommand{\bra}[1]{\langle #1|} $
$ \newcommand{\ket}[1]{|#1\rangle} $
$ \newcommand{\braket}[2]{\langle #1|#2\rangle} $
$ \newcommand{\dot}[2]{ #1 \cdot #2} $
$ \newcommand{\biginner}[2]{\left\langle #1,#2\right\rangle} $
$ \newcommand{\mymatrix}[2]{\left( \begin{array}{#1} #2\end{array} \right)} $
$ \newcommand{\myvector}[1]{\mymatrix{c}{#1}} $
$ \newcommand{\myrvector}[1]{\mymatrix{r}{#1}} $
$ \newcommand{\mypar}[1]{\left( #1 \right)} $
$ \newcommand{\mybigpar}[1]{ \Big( #1 \Big)} $
$ \newcommand{\sqrttwo}{\frac{1}{\sqrt{2}}} $
$ \newcommand{\dsqrttwo}{\dfrac{1}{\sqrt{2}}} $
$ \newcommand{\onehalf}{\frac{1}{2}} $
$ \newcommand{\donehalf}{\dfrac{1}{2}} $
$ \newcommand{\hadamard}{ \mymatrix{rr}{ \sqrttwo & \sqrttwo \\ \sqrttwo & -\sqrttwo }} $
$ \newcommand{\vzero}{\myvector{1\\0}} $
$ \newcommand{\vone}{\myvector{0\\1}} $
$ \newcommand{\vhadamardzero}{\myvector{ \sqrttwo \\  \sqrttwo } } $
$ \newcommand{\vhadamardone}{ \myrvector{ \sqrttwo \\ -\sqrttwo } } $
$ \newcommand{\myarray}[2]{ \begin{array}{#1}#2\end{array}} $
$ \newcommand{\X}{ \mymatrix{cc}{0 & 1 \\ 1 & 0}  } $
$ \newcommand{\Z}{ \mymatrix{rr}{1 & 0 \\ 0 & -1}  } $
$ \newcommand{\Htwo}{ \mymatrix{rrrr}{ \frac{1}{2} & \frac{1}{2} & \frac{1}{2} & \frac{1}{2} \\ \frac{1}{2} & -\frac{1}{2} & \frac{1}{2} & -\frac{1}{2} \\ \frac{1}{2} & \frac{1}{2} & -\frac{1}{2} & -\frac{1}{2} \\ \frac{1}{2} & -\frac{1}{2} & -\frac{1}{2} & \frac{1}{2} } } $
$ \newcommand{\CNOT}{ \mymatrix{cccc}{1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0} } $
$ \newcommand{\norm}[1]{ \left\lVert #1 \right\rVert } $

<h2>Grover's Search: Implementation</h2>

Now we will consider how to implement Grover's search. Let's recall the whole algorithm.

We are given $n$ qubits. At the beginning we apply Hadamard to each qubit, so we put our quantum state into superposition. The amplitude of each basis state $ \ket{0 \cdots 0}, \ldots, \ket{1 \cdots 1} $ is set to $ \frac{1}{\sqrt{N}} $. After that we iterate the following algorithm for several times:
<ul>
    <li>Make a query: apply the oracle operator - it flips the sign of the amplitude of the state that corresponds to the marked element.</li>
    <li>Inversion: apply the inversion matrix - the amplitude of each state is reflected over the mean of all amplitudes.</li>
</ul>

<h3>Query operator</h3>

Suppose that there exists a function $f$ with the following properties:

\begin{align*}
f(x)&=1 &\mbox{ if $x$ is marked}\\
f(x)&=0 &\mbox{ otherwise}
\end{align*}

Such a function can be implemented reversibly by an operator $U_f$ as follows:

<img src="../images/foperator.png" width = "30%">

Now assume that we have access to such an operator $U_f$. An operator which flips the sign of the amplitude of the state corresponding to the marked state $x$ can be constructed using <font color="blue">phase-kickback</font>. Let $y=\ket{-}$ and investigate what happens when we apply $U_f$.

\begin{align*}
U_f \ket{x}\ket{-} &= U_f \ket{x}  \frac{1}{\sqrt{2}} \mypar{ \ket{0}-\ket{1} }\\
&= \frac{1}{\sqrt{2}} (U_f\ket{x}\ket{0}-U_f\ket{x}\ket{1}) \\
&= \frac{1}{\sqrt{2}} (\ket{x}\ket{f(x)\oplus 0} - \ket{x}\ket{f(x)\oplus 1}) \\
&= \ket{x} \frac{1}{\sqrt{2}}\mypar{ \ket{f(x)}-\ket{f(x)\oplus 1}  } \\
&= \ket{x} (-1)^{f(x)} \frac{1}{\sqrt{2}} \mypar{ \ket{0}-\ket{1} }\\
&= (-1)^{f(x)} \ket{x} \ket{-}
\end{align*}

If $f(x)=1$, then its amplitude is flipped and if $f(x)=0$, its amplitude does not change. This is what we want!

<h3>Task 1</h3>


Let $N=4$. Implement the query phase and check the unitary matrix for the query operator. Note that we are interested in the top-left $4 \times 4$ part of the matrix since the remaining parts are due to the ancilla qubit.

First run the following cell to load operator $U_f$. You can access it via:

   f(circuit,qreg).

In [ ]:
%run f.py

Now let's use phase kickback to flip the sign of the marked element:

<ul>
    <li>Set ancilla qubit to $\ket{-}$ by applying X and H.</li>
    <li>Apply operator $U_f$ by calling function $f$
    <li>Set ancilla qubit back.</li>
</ul>

(Can you guess the marked element by looking at the unitary matrix?)

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer

qreg = QuantumRegister(3)
creg = ClassicalRegister(3)

mycircuit = QuantumCircuit(qreg,creg)


#
#Your code here
#


job = execute(mycircuit,Aer.get_backend('unitary_simulator'))
u=job.result().get_unitary(mycircuit,decimals=3)
for i in range(len(u)):
    s=""
    for j in range(len(u)):
        val = str(u[i][j].real)
        while(len(val)<5): val  = " "+val
        s = s + val
    print(s)
    

mycircuit.draw(output='mpl')

<a href="B90_Grovers_Search_Implementation_Solutions.ipynb#task1">click for our solution</a>

<h3>Inversion operator </h3>



To implement the inversion (diffusion) operation, we will need additional (ancilla) qubit. This is how we implement the diffusion operator:

<ul>
    <li>Set the ancilla qubit to $\ket{-}$ by applying X and H.</li>
    <li>Apply H to all qubits other than the ancilla.</li>
    <li>Apply X to all qubits other than the ancilla.</li>
    <li>Apply multiple controlled NOT operator, where the ancilla qubit is target and all other qubits are used for controlling.</li>
    <li>Apply X to all qubits other than the ancilla.</li>
    <li>Apply X to the ancilla qubit.</li>
    <li>Apply H to all qubits other than the ancilla.</li>
    <li>Set ancilla qubit back.</li>
</ul>

Now let's try to understand why these gates are chosen. Let's recall the inversion operator:

$$ 2 \mymatrix{ccc}{
    \frac{1}{N}  & \cdots & \frac{1}{N} \\ 
    \vdots & \ddots & \vdots \\
    \frac{1}{N}  & \cdots & \frac{1}{N} \\ 
    } 
- I . $$


This operator is also called the <font color="blue"> diffusion operator</font>. 

Recall that the diffusion operator can be expressed as $D=2\ket{u}\bra{u}-I$ where $\ket{u}=H^{\otimes n}\ket{0^n}$ is the equal superposition vector.

We will simply denote $\ket{0^n}$ by $\ket{\mathbf{0}}$.

Hence, equivalently 

\begin{align*}
D=2\ket{u}\bra{u}-I &= 2(H^{\otimes n}\ket{\mathbf{0}}\bra{\mathbf{0}}H^{\otimes n}-I) \\
&= 2(H^{\otimes n}\ket{\mathbf{0}}\bra{\mathbf{0}}H^{\otimes n}-H^{\otimes n}H^{\otimes n})\\
&=H^{\otimes n} (2\ket{\mathbf{0}}\bra{\mathbf{0}}H^{\otimes n}-H^{\otimes n}) \\
&=H^{\otimes n} (2\ket{\mathbf{0}}\bra{\mathbf{0}}-I)H^{\otimes n}
\end{align*}

<i>This is why we have H gates at the beginning and at the end</i>

Now let us look at the effect of applying $2\ket{\mathbf{0}}\bra{\mathbf{0}}-I$ to any arbitrary state.

$(2\ket{\mathbf{0}}\bra{\mathbf{0}}-I) \ket{x} = 2\ket{\mathbf{0}}\braket{\mathbf{0}}{x}-\ket{x} .$

If $\ket{x}=\ket{\mathbf{0}}$, since $\braket{\mathbf{0}}{\mathbf{0}}=1$, then $$2\ket{\mathbf{0}}\braket{\mathbf{0}}{\mathbf{0}}-\ket{\mathbf{0}} = 2\ket{\mathbf{0}}-\ket{\mathbf{0}} = \ket{\mathbf{0}}$$.

If $\ket{x}\neq \ket{\mathbf{0}}$, since $\braket{\mathbf{0}}{x}=0$ then $$2\ket{\mathbf{0}}\braket{\mathbf{0}}{x}-\ket{x}= 2\ket{\mathbf{0}}\cdot 0 -\ket{x} = -\ket{x}$$ .

The effect of $2\ket{\mathbf{0}}\bra{\mathbf{0}}-I$  is flipping the amplitude of any state except $\ket{\mathbf{0}}$.

Now let's see how we can implement this operator. Let's define function $g$ as follows and let $U_g$ be the corresponding operator. 

\begin{align*}
g(x)&=0 &\mbox{ if $x$ is $\ket{\mathbf{0}}$ }\\
g(x)&=1 &\mbox{ otherwise},
\end{align*}



Let's set ancilla qubit to state $\ket{-}$ and apply operator $U_g$.
\begin{align*}
U_g \ket{x}\ket{-} &= (-1)^{g(x)} \ket{x} \ket{-}.
\end{align*}


Note that $U_g$ flips the amplitudes of the states other than $\ket{\mathbf{0}}$ and exactly implements $2\ket{\mathbf{0}}\bra{\mathbf{0}}-I$.

How to implement $U_g$?

Apply a not operator to the ancilla qubit when all other qubits are in state 0. Now the output is 1 when all qubits are in state 0.

<i> That's why we have x and multiple controlled not gates. (Recall the Controlled Operations notebook.) </i>

Aplly a not operator to the ancilla so that the output is 0 when all qubits are in state 0, that is $g(\mathbf{x})=0 \iff x = \ket{\mathbf{0}}$.

<i>Therefore we apply x-gate to the ancila qubit at the end. </i>

<h3>Task 2</h3>

Let $N=4$. Implement the inversion operator and check whether you obtain the following matrix:

$\mymatrix{cccc}{-0.5 & 0.5 & 0.5 & 0.5 \\ 0.5 & -0.5 & 0.5 & 0.5 \\ 0.5 & 0.5 & -0.5 & 0.5 \\ 0.5 & 0.5 & 0.5 & -0.5}$.

In [ ]:
def inversion(circuit,quantum_reg):
    
#don't implement the first and last steps in which ancilla qubit is set


#
# your code is here
#

Below you can check the matrix of your inversion operator and how the circuit looks like. We are interested in the top-left $4 \times 4$ part of the matrix, the remaining parts are because we used ancilla qubit.

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer

qreg1 =  QuantumRegister(3)
creg1 = ClassicalRegister(3)

mycircuit1 = QuantumCircuit(qreg1,creg1)

#set ancilla qubit
mycircuit1.x(qreg1[2])
mycircuit1.h(qreg1[2])
    
inversion(mycircuit1,qreg1)

#set ancilla qubit back
mycircuit1.h(qreg1[2])
mycircuit1.x(qreg1[2])


job = execute(mycircuit1,Aer.get_backend('unitary_simulator'))
u=job.result().get_unitary(mycircuit1,decimals=3)
for i in range(len(u)):
    s=""
    for j in range(len(u)):
        val = str(u[i][j].real)
        while(len(val)<5): val  = " "+val
        s = s + val
    print(s)
    
mycircuit1.draw(output='mpl')

<a href="B90_Grovers_Search_Implementation_Solutions.ipynb#task2">click for our solution</a>

<h3>Task 3: Testing Grover's search</h3>

Now we are ready to test our operations and run Grover's search. Suppose that there are 4 elements in the list and try to find the marked element.

First run the following cell to load operator $U_f$. You can access it via:
   f(circuit,qreg)
    
Which state do you observe the most? (Note that the last qubit is the ancilla and it is shared by the query and the inversion operators)

In [ ]:
%run f.py

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer

qreg = QuantumRegister(3)
creg = ClassicalRegister(3)

mycircuit = QuantumCircuit(qreg,creg)

#Grover


#initial step - equal superposition
#
#your code here
#

#set ancilla
#
#your code here
#



#change the number of iterations
iterations=1

#Grover's iterations.
#
#
#
#
   
#set ancilla
#
#your code here
#

mycircuit.measure(qreg,creg)

job = execute(mycircuit,Aer.get_backend('qasm_simulator'),shots=10000)
counts = job.result().get_counts(mycircuit)

# print the reverse of the outcome
for outcome in counts:
    reverse_outcome = ''
    for i in outcome:
        reverse_outcome = i + reverse_outcome
    print(reverse_outcome,"is observed",counts[outcome],"times")

mycircuit.draw(output='mpl')

<a href="B90_Grovers_Search_Implementation_Solutions.ipynb#task3">click for our solution</a>

<h3>Task 4 (Optional, challenging)</h3>

Implement the inversion operation for $n=3$ ($N=8$). This time you will need 5 qubits - 3 for the operation, 1 for ancilla, and one more qubit to implement not gate controlled by three qubits.

In the implementation the ancilla qubit will be qubit 3, while qubits for control are 0, 1 and 2; qubit 4 is used for th multiple control operation. As a result you should obtain the following values in the top-left $8 \times 8$ entries:

$\mymatrix{cccccccc}{-0.75 & 0.25 & 0.25 & 0.25 & 0.25 & 0.25 & 0.25 & 0.25 \\ 0.25 & -0.75 & 0.25 & 0.25 & 0.25 & 0.25 & 0.25 & 0.25 \\ 0.25 & 0.25 & -0.75 & 0.25 & 0.25 & 0.25 & 0.25 & 0.25 \\ 0.25 & 0.25 & 0.25 & -0.75 & 0.25 & 0.25 & 0.25 & 0.25 \\ 0.25 & 0.25 & 0.25 & 0.25 & -0.75 & 0.25 & 0.25 & 0.25 \\ 0.25 & 0.25 & 0.25 & 0.25 & 0.25 & -0.75 & 0.25 & 0.25 \\ 0.25 & 0.25 & 0.25 & 0.25 & 0.25 & 0.25 & -0.75 & 0.25 \\ 0.25 & 0.25 & 0.25 & 0.25 & 0.25 & 0.25 & 0.25 & -0.75}$.

In [ ]:
def big_inversion(circuit,quantum_reg):
    
    
#
# your code is here
#


Below you can check the matrix of your inversion operator. We are interested in the top-left $8 \times 8$ part of the matrix, the remaining parts are because of additional qubits.

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer

big_qreg2 =  QuantumRegister(5)
big_creg2 = ClassicalRegister(5)

big_mycircuit2 = QuantumCircuit(big_qreg2,big_creg2)

#set ancilla
big_mycircuit2.x(big_qreg2[3])
big_mycircuit2.h(big_qreg2[3])
    
big_inversion(big_mycircuit2,big_qreg2)

#set ancilla back
big_mycircuit2.h(big_qreg2[3])
big_mycircuit2.x(big_qreg2[3])

job = execute(big_mycircuit2,Aer.get_backend('unitary_simulator'))
u=job.result().get_unitary(big_mycircuit2,decimals=3)
for i in range(8):
    s=""
    for j in range(8):
        val = str(u[i][j].real)
        while(len(val)<6): val  = " "+val
        s = s + val
    print(s)

<a href="B90_Grovers_Search_Implementation_Solutions.ipynb#task4">click for our solution</a>

<h3>Task 5: Testing Grover's search for 8 elements (Optional, challenging)</h3>

Now we will test Grover's search on 8 elements.

First run the following cell to load operator $U_f$. You can access it via:

You can access the operator $U_{f_8}$ by f_8(circuit,qreg)
    
Which state do you observe the most?  (Note that the last two qubits are extra qubits.)

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer

qreg8 =  QuantumRegister(5)
creg8 = ClassicalRegister(5)

mycircuit8 = QuantumCircuit(qreg8,creg8)


#
#Your code here
#

job = execute(mycircuit8,Aer.get_backend('qasm_simulator'),shots=10000)
counts8 = job.result().get_counts(mycircuit8)
# print the reverse of the outcome
for outcome in counts8:
    reverse_outcome = ''
    for i in outcome:
        reverse_outcome = i + reverse_outcome
    print(reverse_outcome,"is observed",counts8[outcome],"times")

mycircuit8.draw(output='mpl')

<a href="B90_Grovers_Search_Implementation_Solutions.ipynb#task5">click for our solution</a>

Further links:

http://quantumgazette.blogspot.com/2017/12/grovers-algorithm-for-unstructured.html

http://twistedoakstudios.com/blog/Post2644_grovers-quantum-search-algorithm